<a href="https://colab.research.google.com/github/HuangJin-De/Machine-Learning-in-Atmospheric-Thermodynamics/blob/master/final/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! rm -rf Machine-Learning-in-Atmospheric-Thermodynamics
! git clone https://github.com/HuangJin-De/Machine-Learning-in-Atmospheric-Thermodynamics.git

Cloning into 'Machine-Learning-in-Atmospheric-Thermodynamics'...
remote: Enumerating objects: 691, done.
remote: Counting objects: 100% (261/261), done.
remote: Compressing objects: 100% (138/138), done.
remote: Total 691 (delta 129), reused 242 (delta 118), pack-reused 430
Receiving objects: 100% (691/691), 249.37 MiB | 16.66 MiB/s, done.
Resolving deltas: 100% (287/287), done.


In [3]:
import numpy as np
import matplotlib.pyplot as plt

path='Machine-Learning-in-Atmospheric-Thermodynamics/final/'
filen=path+'/data/inver_data.dat'

nz=101
nvar=3
data=np.fromfile(filen,dtype=np.float32)
data=data.reshape(-1,2,nvar,nz)
ind=np.arange(data.shape[0])
np.random.shuffle(ind)
data=data[ind,:,:,:]

#print(data.shape)

temp=np.zeros([data.shape[0],99])
inver_era=np.argmax(data[:,0,2,0:99],1)
indx=np.arange(temp.shape[0])
indy=inver_era
temp[indx,indy]=1
temp[:,0]=0
mask_era=np.copy(temp)

#print(np.sum(mask_era,0),np.sum(mask_era))
#print(np.sum(mask_era))

temp=np.zeros([data.shape[0],99])
inver_snd=np.argmax(data[:,1,2,0:99],1)
indx=np.arange(temp.shape[0])
indy=inver_snd
temp[indx,indy]=1
temp[:,0]=0
mask_snd=np.copy(temp)
